In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import re
import csv
import os
import json

In [ ]:
! pip install -q transformers datasets sentencepiece

!pip uninstall -y -q transformers accelerate
!pip install -q transformers accelerate

!pip install -q rouge-score

!pip install -q torchmetrics

!pip install -q git+https://github.com/huggingface/accelerate

!git clone -q https://github.com/huggingface/transformers.git
!cd transformers
!pip install  -q -e.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements

In [ ]:
model_name_or_path = '/content/drive/MyDrive/mt5-pquad/checkpoint-1917'
from transformers import AutoTokenizer, MT5ForConditionalGeneration
tokenizer_test = AutoTokenizer.from_pretrained(model_name_or_path)
model_test = MT5ForConditionalGeneration.from_pretrained(model_name_or_path)

In [ ]:
verses = set()

with open('/content/drive/MyDrive/single_verse.jsonl', 'r') as f:
  data_list = []
  for line in f:
    data = json.loads(line)
    data_list.append(data)

for qa in data_list:
  verses.add(qa['pq_id'])

In [ ]:
ner_path = '/content/drive/MyDrive/quran_named_entities/'

dict_data_list = []

for each in os.listdir(ner_path):
  file_path = '/content/drive/MyDrive/quran_named_entities/' + each
  entity_file = open(file_path, 'r', encoding='utf-8-sig')
  reader = csv.DictReader(entity_file)
  entity_list = list(reader)
  dict_data_list += entity_list

# for dict_data in dict_data_list:
#   if dict_data['verse_id'] in verses:
#     dict_data_list.remove(dict_data)

unique_verses = []

input_data = []
answer = []
context = []
for i in range(0, len(dict_data_list)):
  if dict_data_list[i]['verse_id'] not in verses:
    input_data.append('answer: ' + dict_data_list[i]['answer'].strip() + ' context: ' + dict_data_list[i]['context'])
    answer.append(dict_data_list[i]['answer'])
    context.append(dict_data_list[i]['context'])

    unique_verses.append(dict_data_list[i])

In [ ]:
input_data[2]

In [ ]:
def custom_gen(model, tokenizer, input_text, max_length=50, do_sample=False, temperature=None, num_beams=10, top_k=None, top_p=None, early_stopping=True, num_return_sequences=1):
               input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
               generated_ids = model.generate(input_ids, max_length=max_length, do_sample=do_sample, top_k=top_k, temperature=temperature, num_beams=num_beams)
#generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
               return generated_ids

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_test.to(device)

# Using the custom_gen. Will have to change for our code.
bn_pred_ques = {}
all_generated_ids = []
for prompt in input_data:
    # print(prompt)
    generated_ids = custom_gen(model_test,tokenizer_test,prompt,max_length=50,do_sample=True,top_k=40, top_p=0.95,num_return_sequences=20)
    all_generated_ids.append(generated_ids)
pred = []
pred_ques =[]
for text in all_generated_ids:
    for generated in text:
        generated_text = tokenizer_test.decode(generated,skip_special_tokens=True)
#         print(generated_text)
        pred.append(generated_text)
        bn_pred_ques[prompt] = pred
#     pred_ques.append(pred)
# print(pred)
# for generated in generated_ids:
#     generated_text = tokenizer_test.decode(generated,skip_special_tokens=True)
#     print(generated_text)
#     pred.append(generated_text)
#     bn_pred_ques[prompt] = pred

In [ ]:
print(pred)

NameError: ignored

In [ ]:
# counter = 0
# for each in pred:
#   print(counter)
#   print(each)
#   print("*************")
#   counter+=1

In [ ]:
input_data[6]

'answer: قرآن context: ما اینچنین  قرآن را به درون دلهای مجرمان راه می\u200cدهیم!'

In [ ]:
len(input_data)

8436

write to file

In [ ]:
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

In [ ]:
new_data_list = []
multiple_occurence = []

for i in range(0, len(unique_verses)):
  new_data = {}
  answer = unique_verses[i]['answer'].strip()
  context = unique_verses[i]['context']

  new_data['context'] = context
  new_data['pq_id'] = unique_verses[i]['verse_id']
  new_data['question'] = pred[i]

  if context.count(answer) == 1:
    answers_list = []
    answer_dict = {}
    answer_dict['text'] = answer
    answer_dict['start_char'] = context.find(answer)
    answers_list.append(answer_dict)
    new_data['answers'] = answers_list
    new_data_list.append(new_data)

  else:
    new_data['answers'] = answer
    new_data['freq'] = context.count(answer)
    multiple_occurence.append(new_data)

In [ ]:
len(new_data_list)

3239

In [ ]:
output_path = '/content/drive/MyDrive/automatic_data.jsonl'
dump_jsonl(new_data_list, output_path)

Wrote 3239 records to /content/drive/MyDrive/automatic_data.jsonl


In [ ]:
len(multiple_occurence)

5197

In [ ]:
output_path = '/content/drive/MyDrive/automatic_multiple_occurence.jsonl'
dump_jsonl(multiple_occurence, output_path)

Wrote 5197 records to /content/drive/MyDrive/automatic_multiple_occurence.jsonl
